# Tutorial III: Low rank, arbitrary basis molecular simulations

## Important note about origins of this method

What follows in this tutorial is currently unpublished. It will be described in detail in an upcoming paper by (alphabetical) Ryan Babbush, Garnet Kin-Lic Chan, Jarrod McClean, Austin Minnich, Mario Motta, and Erika Ye.

## Low rank decomposition of the Coulomb operator

In Tutorial I we discussed methods for very compiling single-particle basis transformations of fermionic operators in $O(N)$ depth on a linearly connected architecture. We looked at the particular example of simulating a free fermion model by using Bogoliubov transformations to diagonalize the model.

In Tutorial II we discussed methods for compiling Trotter steps of electronic structure Hamiltonian in $O(N)$ depth on a linearly connected architecture when expressed in a basis diagonalizing the Coulomb operator so that
\begin{equation}
H = \sum_{pq} T_{pq} a^\dagger_p a_q + \sum_{pq} V_{pq} a^\dagger_p a_p a^\dagger_q a_q.
\end{equation}

Here we will discuss how both of those techniques can be combined, along with some insights from electronic structure, in order to simulate arbitrary basis molecular Hamiltonians taking the form
\begin{equation}
H = \sum_{pq} T_{pq} a^\dagger_p a_q + \sum_{pqrs} V_{pqrs} a^\dagger_p a_q a^\dagger_r a_s
\end{equation}
in depth scaling only as $O(N^2)$ on a linear array of qubits. First, we note that the one-body part of the above expression is easy to simulate using the techniques introduced in Tutorial I. Thus, the real challenge is to simulate the two-body part of the operator.

We begin with the observation that the rank-4 tensor $V$, with the values $V_{pqrs}$ representing the coefficient of $a^\dagger_p a_q a^\dagger_r a_s$ can be flattened into an $N^2 \times N^2$ array by making $p,q$ one index and $r,s$ the other. This is the electronic repulsion integral (ERI) matrix in chemist notation. We will refer to the ERI matrix as $W$. By diagonalizing $W$, one obtains $W g_\ell = \lambda_\ell g_\ell$ where the eigenvector $g_\ell$ is a vector of dimension $N^2$. If we reshape $g_\ell$ into an $N \times N$ vector, we realize that
\begin{equation}
\sum_{pqrs} V_{pqrs} a^\dagger_p a_q a^\dagger_r a_s = \sum_{\ell=0}^{L-1} \lambda_\ell \left(\sum_{pq} \left[g_{\ell}\right]_{pq} a^\dagger_p a_q\right)^2.
\end{equation}
This is related to the concept of density fitting in electronic structure, which is often accomplished using a Cholesky decomposition. It is fairly well known in the quantum chemistry community that the ERI matrix is positive semi-definite and despite having linear dimension $N^2$, has rank of only $L = O(N)$. Thus, the eigenvalues $\lambda_\ell$ are positive and there are only $O(N)$ of them.

Next, we diagonalize the $g_\ell$ matrices so that
$u_\ell g_\ell u_\ell^\dagger = \textrm{diag}(f_\ell)$ where $f_\ell$ is a length $N$ vector and $\textrm{diag}(f_\ell)$ is a diagonal $N \times N$  matrix with the entries of $f_\ell$ on the diagonal. Then,
\begin{equation}
\sum_{\ell=0}^{L-1} \lambda_\ell \left(\sum_{pq} \left[g_{\ell}\right]_{pq} a^\dagger_p a_q\right)^2 =
\sum_{\ell=0}^{L-1} \lambda_\ell \left(u_\ell \left(\sum_{p} f_{\ell p} a^\dagger_p a_p\right) u_\ell^\dagger\right)^2 = \sum_{\ell=0}^{L-1} \lambda_\ell \left(u_\ell \left(\sum_{p} f_{\ell p} a^\dagger_p a_p\right) u_\ell^\dagger u_\ell \left(\sum_{p} f_{\ell p} a^\dagger_p a_p\right) u_\ell^\dagger\right)
= \sum_{\ell=0}^{L-1} \lambda_\ell u_\ell \left(\sum_{pq} f_{\ell p} f_{\ell q} a^\dagger_p a_p a^\dagger_q a_q\right) u_\ell^\dagger
\end{equation}
Note that $u_\ell$ is a basis transformation matrix since it is the eigenvectors of a one-particle operator. Then, if $R(u_\ell)$ is the circuit realizing the Bogoliubov transformation associated with the basis change matrix $u_\ell$, we see that we can simulate a Trotter step under the arbitrary basis two-body operator as
\begin{equation}
\prod_{\ell=0}^{L-1} R(u_\ell) \exp\left(-i\sum_{pq} f_{\ell p} f_{\ell q} a^\dagger_p a_p a^\dagger_q a_q\right) R(u_\ell^\dagger)
\end{equation}
where we note that the operator in the exponential take the form of a diagonal Coulomb operator. Since we can implement the $R(u)$ circuits in $O(N)$ depth (see Tutorial I) and we can implement Trotter steps under diagonal Coulomb operators in $O(N)$ layers of gates (see Tutorial II) we see that we can implement Trotter steps under arbitrary basis electronic structure Hamiltionians in $O(N^2)$ depth, and all on a linearly connected device. This is a big improvement over the usual way of doing things, which would lead to no less than $O(N^5)$ depth!

Note that these techniques are also applicable to realizing evolution under other two-body operators, such as the generator of unitary coupled cluster. Note that one can create variational algorithms where a variational parameter specifies the rank at which to truncate the $\lambda_\ell$.

## Example implementation: Trotter steps of LiH in molecular orbital basis

We will now use these techniques to implement Trotter steps for an actual molecule. We will focus on LiH at equilibrium geometry, since integrals for that system are provided with every OpenFermion installation. However, by installing [OpenFermion-PySCF](https://github.com/quantumlib/OpenFermion-PySCF) or [OpenFermion-Psi4](https://github.com/quantumlib/OpenFermion-Psi4) one can use these techniques for any molecule at any geometry. We will generate LiH in an active space consisting of 4 qubits. First, we obtain the Hamiltonian as an InteractionOperator.

In [1]:
import openfermion

# Load saved file for LiH.
diatomic_bond_length = 1.45
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1

# Set Hamiltonian parameters.
active_space_start = 1
active_space_stop = 3

# Generate and populate instance of MolecularData.
molecule = openfermion.hamiltonians.MolecularData(geometry, basis, multiplicity, description="1.45")
molecule.load()

# Get the Hamiltonian in an active space.
molecular_hamiltonian = molecule.get_molecular_hamiltonian(
    occupied_indices=range(active_space_start),
    active_indices=range(active_space_start, active_space_stop))
print(openfermion.transforms.get_fermion_operator(molecular_hamiltonian))

-6.7698132180879735 [] +
-0.7952726864779313 [0^ 0] +
0.24889540266275176 [0^ 1^ 1 0] +
-0.02307282640154995 [0^ 1^ 1 2] +
-0.023072826401549944 [0^ 1^ 3 0] +
0.005865992881900444 [0^ 1^ 3 2] +
0.04614563473199314 [0^ 2] +
0.005865992881900456 [0^ 2^ 0 2] +
0.11412688446849813 [0^ 2^ 2 0] +
-0.02307282640154995 [0^ 3^ 1 0] +
0.005865992881900456 [0^ 3^ 1 2] +
0.11412688446849813 [0^ 3^ 3 0] +
0.0027487522157917266 [0^ 3^ 3 2] +
0.24889540266275176 [1^ 0^ 0 1] +
-0.02307282640154995 [1^ 0^ 0 3] +
-0.023072826401549944 [1^ 0^ 2 1] +
0.005865992881900444 [1^ 0^ 2 3] +
-0.7952726864779313 [1^ 1] +
-0.02307282640154995 [1^ 2^ 0 1] +
0.005865992881900456 [1^ 2^ 0 3] +
0.11412688446849813 [1^ 2^ 2 1] +
0.0027487522157917266 [1^ 2^ 2 3] +
0.04614563473199314 [1^ 3] +
0.005865992881900456 [1^ 3^ 1 3] +
0.11412688446849813 [1^ 3^ 3 1] +
0.04614563473199324 [2^ 0] +
0.11412688446849814 [2^ 0^ 0 2] +
0.005865992881900458 [2^ 0^ 2 0] +
-0.023072826401549958 [2^ 1^ 1 0] +
0.11412688446849814 [2^ 1^ 

We see from the above output that this is a fairly complex Hamiltonian already. Next we will use the `simulate_trotter` function from Tutorial I, but this time using a different type of Trotter step associated with these low rank techniques. To keep this circuit very short for pedagogical purposes we will force a truncation of the eigenvalues $\lambda_\ell$ at a predetermined value of `final_rank`. While we also support a canned `LOW_RANK` option for the Trotter steps, in order to pass this value of `final_rank` we will instantiate a custom Trotter algorithm type.

In [2]:
import cirq
import openfermioncirq

# Trotter step paramaters.
time = 1.
final_rank = 2

# Initialize circuit qubits in a line.
n_qubits = openfermion.utils.count_qubits(molecular_hamiltonian)
qubits = cirq.LineQubit.range(n_qubits)

# Compile the low rank Trotter step using OpenFermion-Cirq.
custom_algorithm = openfermioncirq.trotter.LowRankTrotterAlgorithm(final_rank=final_rank)
circuit = cirq.Circuit.from_ops(
    openfermioncirq.trotter.simulate_trotter(
            qubits, molecular_hamiltonian,
            time=time, omit_final_swaps=True,
            algorithm=custom_algorithm))

# Print circuit.
cirq.DropNegligible().optimize_circuit(circuit)
print(circuit.to_text_diagram(transpose=True))

0          1           2           3
│          │           │           │
×ᶠ─────────×ᶠ          │           │
│          │           │           │
│          │           ×ᶠ──────────×ᶠ
│          │           │           │
│          ×ᶠ──────────×ᶠ          │
│          │           │           │
XXYY───────XXYY^0.0229 │           │
│          │           │           │
×ᶠ─────────×ᶠ          XXYY────────XXYY^0.0229
│          │           │           │
│          │           ×ᶠ──────────×ᶠ
│          │           │           │
│          ×ᶠ──────────×ᶠ          Z^0.178
│          │           │           │
Z^0.054    Z^0.054     Z^0.178     │
│          │           │           │
│          #2──────────YXXY        │
│          │           │           │
│          │           #2──────────YXXY^-0.14
│          │           │           │
#2─────────YXXY^0.14   │           │
│          │           │           │
│          Z           │           │
│          │           │           │
│          

We were able to print out the circuit this way but forcing `final_rank` of 2 is not very accurate. In the cell below, we compile the Trotter step with full rank so $L = N^2$ and depth is actually $O(N^3)$ and repeat the Trotter step multiple times to show that it actually converges to the correct result. Since we are not forcing the rank truncation we can use the built-in `LOW_RANK` Trotter step type.

In [3]:
# Initialize a random initial state.
import numpy
random_seed = 8317
numpy.random.seed(random_seed)
initial_state = numpy.random.randn(2 ** n_qubits).astype(numpy.complex64, copy=False)
initial_state /= numpy.linalg.norm(initial_state)

# Numerically compute the correct circuit output.
import scipy
hamiltonian_sparse = openfermion.get_sparse_operator(molecular_hamiltonian)
exact_state = scipy.sparse.linalg.expm_multiply(-1j * time * hamiltonian_sparse, initial_state)

# Trotter step paramaters.
time = 1.
n_steps = 2

# Compile the low rank Trotter step using OpenFermion-Cirq.
qubits = cirq.LineQubit.range(n_qubits)
circuit = cirq.Circuit.from_ops(
    openfermioncirq.trotter.simulate_trotter(
            qubits, molecular_hamiltonian,
            time=time, n_steps=n_steps, algorithm=openfermioncirq.trotter.LOW_RANK))
cirq.DropNegligible().optimize_circuit(circuit)

# Use Cirq simulator to apply circuit.
simulator = cirq.google.XmonSimulator()
result = simulator.simulate(circuit, qubit_order=qubits, initial_state=initial_state)
simulated_state = result.final_state

# Print final fidelity.
fidelity = abs(numpy.dot(simulated_state, numpy.conjugate(exact_state))) ** 2
print('Fidelity with exact result is {}. Circuit follows'.format(fidelity))

# Print circuit.
cirq.DropNegligible().optimize_circuit(circuit)
print(circuit.to_text_diagram(transpose=True))

Fidelity with exact result is 0.9999705131395995. Circuit follows
0          1              2              3
│          │              │              │
×ᶠ─────────×ᶠ             │              │
│          │              │              │
│          │              ×ᶠ─────────────×ᶠ
│          │              │              │
│          ×ᶠ─────────────×ᶠ             │
│          │              │              │
XXYY───────XXYY^0.0115    │              │
│          │              │              │
×ᶠ─────────×ᶠ             XXYY───────────XXYY^0.0115
│          │              │              │
│          │              ×ᶠ─────────────×ᶠ
│          │              │              │
│          ×ᶠ─────────────×ᶠ             Z^0.0891
│          │              │              │
Z^0.027    Z^0.027        Z^0.0891       │
│          │              │              │
│          #2─────────────YXXY           │
│          │              │              │
│          │              #2─────────────YXXY^-0.14
│  